# Dev and Test notebook

## Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import skimage.external.tifffile as tiff

from common import Statistics, dataset_source
from resources.conv_learner import *
from resources.plots import *
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

## Allowing normalization according to both dataset (source) and class

In [3]:
PATH = "datasets/yeast_v5.3/"
data_path = Path(PATH)

In [4]:
stats_name = "yeast_v5.3_per_class.dict"
classes = Statistics.source_class(data_path)

train_val = zip(classes['train'], classes['val'])
# Xtest = zip(classes['test'])
 
main_stats = Statistics.per_class(train_val, save_name=stats_name)
# test_stats = Statistics.per_class(Xtest, save_name=stats_name)

working on: datasets\yeast_v5.3\train\01_mfb1KO
working on: datasets\yeast_v5.3\val\01_mfb1KO
working on: datasets\yeast_v5.3\train\01_mmm1KO
working on: datasets\yeast_v5.3\val\01_mmm1KO
working on: datasets\yeast_v5.3\train\01_mmr1KO
working on: datasets\yeast_v5.3\val\01_mmr1KO
working on: datasets\yeast_v5.3\train\01_num1KO
working on: datasets\yeast_v5.3\val\01_num1KO
working on: datasets\yeast_v5.3\train\01_WT
working on: datasets\yeast_v5.3\val\01_WT
working on: datasets\yeast_v5.3\train\02_mfb1KO
working on: datasets\yeast_v5.3\val\02_mfb1KO
working on: datasets\yeast_v5.3\train\02_mmr1KO
working on: datasets\yeast_v5.3\val\02_mmr1KO
working on: datasets\yeast_v5.3\train\02_mmr1KO-mfb1KO
working on: datasets\yeast_v5.3\val\02_mmr1KO-mfb1KO
working on: datasets\yeast_v5.3\train\02_WT
working on: datasets\yeast_v5.3\val\02_WT


In [5]:
for keys in main_stats.keys():
    print(f"{keys}: \t \t \t {main_stats[keys]}")

01_mfb1KO: 	 	 	 (array([0.02112, 0.00447]), array([0.00145, 0.00144]))
01_mmm1KO: 	 	 	 (array([0.021  , 0.00422]), array([0.00146, 0.00192]))
01_mmr1KO: 	 	 	 (array([0.02116, 0.00486]), array([0.00156, 0.00197]))
01_num1KO: 	 	 	 (array([0.02098, 0.00502]), array([0.00146, 0.00212]))
01_WT: 	 	 	 (array([0.0211 , 0.00456]), array([0.00151, 0.00132]))
02_mfb1KO: 	 	 	 (array([0.00793, 0.00491]), array([0.00076, 0.00171]))
02_mmr1KO: 	 	 	 (array([0.00798, 0.005  ]), array([0.00075, 0.002  ]))
02_mmr1KO-mfb1KO: 	 	 	 (array([0.0079 , 0.00485]), array([0.00075, 0.00159]))
02_WT: 	 	 	 (array([0.00794, 0.00479]), array([0.00076, 0.00151]))


In [6]:
def read_dirsX(path, folder):
    '''
    Fetches name of all files in path in long form, and labels associated by extrapolation of directory names. 
    '''
    lbls, fnames, all_lbls = [], [], [] #(!) Need to modify here for z lable
    full_path = os.path.join(path, folder)
    for lbl in sorted(os.listdir(full_path)):
        if lbl not in ('.ipynb_checkpoints', '.DS_Store'):
            all_lbls.append(lbl)
            for fname in os.listdir(os.path.join(full_path, lbl)):
                if fname not in ('.DS_Store'):
                    fnames.append(os.path.join(folder, lbl, fname))
                    lbls.append(lbl)
    return fnames, lbls, all_lbls

In [21]:
path = PATH
folder = 'val'
fnames, lbls, all_lbls = read_dirsX(path, folder)
    
lbls2classes = {l: l.split('_')[1] for l in all_lbls} # (!) dict mapping lbls (folders) to classes (genotype)
u_classes = list(dict.fromkeys(list(lbls2classes.values()))) # (!) get unique classes

d = {}
if d == {}:
    d_cls = {label: idx for idx, label in enumerate(u_classes)}
    d_src = {label_: idx_ for idx_, label_ in enumerate(lbls2classes)}
    
#     for idx, label in enumerate(u_classes):
#         d_cls[label] = idx
#     for idx_, label_ in enumerate(lbls2classes):
#         d_src[label_] = idx_

    # (!) populating d with lbls: class, class_idx   
    for key, value in lbls2classes.items():
        d[key] = [d_src[key], d_cls[value], lbls2classes[key]] 

src_idxs  = [d[lbl][0] for lbl in lbls] # (!) using d to generate class-lables for each image
cls_idxs  = [d[lbl][1] for lbl in lbls] # (!) using d to generate class-lables for each image

src_idx_arr = np.array(src_idxs, dtype=int)
cls_idx_arr = np.array(cls_idxs, dtype=int)

In [22]:
cls_idx_arr.shape

(315,)

In [ ]:
print(d)
print(src_idx_arr)
print(cls_idx_arr)

In [14]:
lbl2indexX = {}
trnX = folder_sourceX(PATH, 'val', lbl2indexX)